# Рынок заведений общественного питания Москвы

Цель: Подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.

Мне доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках. Она носит исключительно справочный характер.

Описание данных
- name — название заведения;
- address — адрес заведения;
- category — категория заведения, например «кафе», «пиццерия» или «кофейня»;
- hours — информация о днях и часах работы;
- lat — широта географической точки, в которой находится заведение;
- lng — долгота географической точки, в которой находится заведение;
- rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
- price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
- avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:


«Средний счёт: 1000–1500 ₽»;


«Цена чашки капучино: 130–220 ₽»;


«Цена бокала пива: 400–600 ₽»;
- middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»:


Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.


Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.


Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт.


- middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:


Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.



Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.


Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт.


- chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):


0 — заведение не является сетевым


1 — заведение является сетевым


- district — административный район, в котором находится заведение, например Центральный административный округ;
- seats — количество посадочных мест.


## Изучу общую информацию о данных

### Подключy библиотеки

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from scipy import stats as st
import math as mth
from plotly import graph_objects as go
import seaborn as sns
import plotly.express as px
import folium
# импортируем карту и маркер
from folium import Map, Marker
# импортируем кластер
from folium.plugins import MarkerCluster
from folium import Map, Marker
from folium import Map, Choropleth
import numpy as np

### Открою файл

In [3]:
data = pd.read_csv('/datasets/dash_visits.csv', sep=',')
display(data.head())

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/dash_visits.csv'

In [ ]:
print(data.info())

In [ ]:
data = pd.read_csv('/datasets/moscow_places.csv', sep=',')

Мы имеем данные о 8 406 заведениях

Количество посадочных мест должно быть целым числом, изменим тип столбца seats 

In [ ]:
data['seats'] = data['seats'].astype(int, errors = 'ignore')

## Предобработка

### Пропущенные значения

In [ ]:
data.isna().sum()

Пропущены значения в столбцах:
- hours (время работы) - 536
- price (категория цен) - 5091
- avg_bill(средняя стоимость заказа) - 4590
- middle_avg_bill(средний чек) - 5257
- middle_coffee_cup(цена одной чашки капучино) - 7871
- seats(количество посадочных мест) - 3611

К сожалению, эти пропуски невозможно обработать. Часы работы, категория цен, средняя стоиомость заказа, количество посадочных мест - это данные, которые могут быть абсолютно разными и не от чего не зависят. Цена одной чашки капучино может быть не указана, так как место не является кофейней.

In [ ]:
data['address'] = data['address'].str.lower()
data['name'] = data['name'].str.lower()

In [ ]:
data[data.duplicated(['address', 'name'])].count()


In [ ]:
data[data.duplicated(['address', 'name'])].drop_duplicates()

In [ ]:
data.duplicated().sum()

Дубликатов нет

Столбец street с названиями улиц

In [ ]:
data['street'] = data.address.str.split(', ').str[1]

Столбец is_24/7 с указанием является ли заведение круглосуточным

In [ ]:
data['is_24/7'] = data['hours'].str.contains('ежедневно, круглосуточно')

Проверим рейтинг на наличие выбросов(он должен находиться в диапазоне от 1 до 5) 

In [ ]:
print(data['rating'].max())

In [ ]:
print(data['rating'].min())

Оценим стоимость чашки кофе на выбросы

In [ ]:
plt.hist(data['middle_coffee_cup']) 

Из графика видно, что ценовая вилка чашки кофе 90 - 370 рублей

In [ ]:
#data = data.dropna(subset=['middle_coffee_cup'])
#print(np.percentile(data['middle_coffee_cup'], [90, 95, 99])) 
print(data['middle_coffee_cup'].quantile(0.9))
print(data['middle_coffee_cup'].quantile(0.95))
print(data['middle_coffee_cup'].quantile(0.99))

Не более чем у 5% пользователей заказ был выше 275 рублей, не более 1% дороже 310 рублей. Можно предположить, что цена чашки кофе не более 310.

In [ ]:
data.loc[data['middle_coffee_cup']<310, 'middle_coffee_cup']

In [ ]:
data['middle_coffee_cup'] = data.loc[data['middle_coffee_cup']<310, 'middle_coffee_cup']

В столбце chain должно быть 2 значения 0 и 1. Проверим это

In [ ]:
print(data['chain'].unique())

Проверим столбец seats с количеством посадочных мест на выбросы и артефакты

In [ ]:
plt.hist(data['seats']) 

In [ ]:
#data = data.dropna(subset=['seats'])
#print(np.percentile(data['seats'], [90, 95, 99]))
print(data['seats'].quantile(0.9))
print(data['seats'].quantile(0.95))
print(data['seats'].quantile(0.99))

Не более чем у 5% заведений количество мест выше 307, не более 1% выше 625 рублей.Приму за достоверную информацию количество мест от 0 рублей до 320.

In [ ]:
data['seats'] = data.loc[data['seats']<307, 'seats']

## Анализ данных

Посмотрим какие категории заведений существуют

In [ ]:
data['category'].unique()

In [ ]:
category_count = data.groupby('category').agg({'name': 'count'})
count = category_count['name']

In [ ]:
fig = go.Figure(data=[go.Pie(labels=category_count.index, values=count)])
fig.update_layout(title='Распределение заведений по категориям') #заголовок
fig.show() 

Больше всего из представленных заведений это кафе и рестораны, также много кофейн. Меньше всего булочных и столовых

Проверим как распределены места между категориями

In [ ]:
seats_median = data.groupby('category').agg({'seats': 'median'}).sort_values(by = 'seats',ascending = False)
count_seats = seats_median['seats']

In [ ]:
data.groupby('category').agg({'seats': 'mean'}).sort_values(by = 'seats',ascending = False)

Среднее и медианное значение сильно отличаются, значит в данных есть выбросы. Так как среднее значение больше медианного, то выбросы это очень большие значения(количества мест). Следовательно, среднее значение будем смотреть по медиане.

In [ ]:
fig = px.bar(seats_median, x=seats_median.index, y='seats', title='Среднее количество мест по заведениям')
fig.update_xaxes(tickangle=45)
fig.show() 

Больше всего мест в ресторанах - 86, а меньше всего в булочных - 50.

Посмотрим как соотносятся сетевые и несетевые заведения

In [ ]:
chain_count = data.groupby('chain').agg({'name': 'count'})
count_chain = chain_count['name']
chain_count

In [ ]:

name= ['несетевые','сетевые']
fig = go.Figure(data=[go.Pie(labels=name, values=count_chain,title='Сетевые/несетевые')])
fig.show() 

Несетевых заведений оказалось больше почти в 2 раза

Рассмотрим подробнее сетевые заведения

In [ ]:

chain_places = data[data['chain'] == 1]
fig = px.histogram(data, # загружаем данные
                   x='category', # указываем столбец с данными для оси X
                   color='chain', # обозначаем категорию для разделения цветом
                   title='Распределение категорий заведений среди сетевых и несетевых', # указываем заголовок
                   nbins=8, # назначаем число корзин
                   barmode='overlay') # выбираем «полупрозрачный» тип отображения столбцов
fig.update_xaxes(title_text='Значение') # подпись для оси X
fig.update_yaxes(title_text='Количество') # подпись для оси Y
fig.show() # выводим график

- Среди сетевых заведений больше всего кафе, ресторанов и кофейн
- Среди несетевых заведений больше всего кафе, ресторанов
- Кафе, рестораны, бары чаще всего относятся к несетевым
- Булочные, пиццерии и кофейни чаще сетевые

In [ ]:
chain_places = data[data['chain'] == 1]
chain_places

In [ ]:
# отфильтруем данные, сгруппируем по городам и посчитаем объявления
data_loc_count = chain_places.groupby('name')[['name']].count()
# переименуем столбец
data_loc_count.columns = ['count']
# отсортируем и оставим 15 лидеров
data_loc_count = data_loc_count.reset_index().sort_values(by='count', ascending=False).head(15)
display(data_loc_count)

Самыми популярными сетями в Москве оказались:
- Шоколадница
- Доминос пицца
- Додо Пицца


Самыми непопулярными сетями оказались:
- CofeFest	
- Буханка	
- Му-Му

In [ ]:
# строим столбчатую диаграмму 
fig = px.bar(data_loc_count.sort_values(by='count', ascending=True), # загружаем данные и заново их сортируем
             x='count', # указываем столбец с данными для оси X
             y='name', # указываем столбец с данными для оси Y
             text='count' # добавляем аргумент, который отобразит текст с информацией
                                # о количестве объявлений внутри столбца графика
            )
# оформляем график
fig.update_layout(title='ТОП-15 сетевых заведений',
                   xaxis_title='Количество заведений',
                   yaxis_title='Название заведения')
fig.show() # выводим график

In [ ]:
top = pd.DataFrame(chain_places.groupby('name').count().sort_values(by='category', ascending=False).head(15)['category']).index
pd.DataFrame(chain_places.query('name in @top').groupby('name')['category'].unique())

Среди Топ-15 больше всего кофейн, кафе и ресторанов, но также есть несколько сетей пиццерий

In [ ]:
data['district'].unique()

Всего есть данные о 9 округах: 
- 'Северный административный округ',
- 'Северо-Восточный административный округ',
-  'Северо-Западный административный округ',
- 'Западный административный округ',
-  'Центральный административный округ',
- 'Восточный административный округ',
- 'Юго-Восточный административный округ',
- 'Южный административный округ',
- 'Юго-Западный административный округ'

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x='district', hue='category', data=data)
plt.xticks(rotation=90)
plt.title('Распределение категорий по районам')
plt.show()

In [ ]:
data.pivot_table(index='district',
                 columns='category',
                 values='name',
                 aggfunc='count',
                 margins=True).drop('All').sort_values('All').drop(columns='All').plot(kind='bar',
                                                                                       stacked=True)

In [ ]:
data.groupby('district')['name'].count().sort_values()

- В центральном округе больше всего заведений
- В Северо-Западном округе меньше всего заведений
- Везде выделяется категории кафе и ресторанов(больше всего заведений)
- Меньше всего булочных

In [ ]:
data.groupby('category')['rating'].mean().sort_values(ascending=False).plot(kind='bar')
plt.title('Средний рейтинг по категориям')
plt.xlabel('Категория')
plt.ylabel('Рейтинг')
plt.show()

- Самый высокий рейтинг у баров, пабов, а самый низкий у быстрого питания
- Рейтинги всех заведений колеблются между 4-5 баллами и не сильно отличаются между собой

In [ ]:
rating_df = data.groupby('district', as_index=False)['rating'].agg('median')
rating_df

Самый высокий рейтинг у Центрального округа

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['category']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=rating_df,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGnBu',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)



# выводим карту
m

На востоке столицы рейтинг ниже

In [ ]:
data.groupby('street')['name'].count().sort_values(ascending = False).head(15).plot(kind='bar',title = 'Количество заведений на топ 15 улицах')
top_street = data.groupby('street')['name'].count().sort_values(ascending = False).head(15).index

Самое большое количество общепита находится на проспекте Мира

In [ ]:
top_st_data = data.query('street in @top_street')
plt.figure(figsize=(20, 10))
sns.countplot(x='street', hue='category', data=top_st_data)
plt.xticks(rotation=90)
plt.show()

На проспекте мира много кафе, ресторанов. Скорее всего это очень оживленная улица.
На МКАДе тоже много кафе, но скорее всего за счет протяженности дороги

In [ ]:
data = data.merge(data.groupby('street')['name'].count().rename('is_alone') == 1, on='street')

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x='is_alone', hue='category', data=data)
plt.xticks(rotation=90)
plt.title('Распределение категорий заведений среди одиночных и не одиночных')
plt.show()

In [ ]:
fig = px.histogram(data, # загружаем данные
                   x='category', # указываем столбец с данными для оси X
                   color='is_alone', # обозначаем категорию для разделения цветом
                   title='Распределение категорий заведений среди одиночных и не одиночных', # указываем заголовок
                   nbins=8, # назначаем число корзин
                   barmode='overlay') # выбираем «полупрозрачный» тип отображения столбцов
fig.update_xaxes(title_text='Значение') # подпись для оси X
fig.update_yaxes(title_text='Количество') # подпись для оси Y
fig.show() # выводим график

- Среди одиночных заведений самыми распространенными являются кафе, рестораны и кофейни(также, как и у не одиночных)
- А вот одну пиццерию редко можно встретить на улице, также как и булочную
- Чаще на одной улице расположено несколько заведений

In [ ]:
data.groupby('is_alone')[['seats','rating','middle_avg_bill']].mean()

In [ ]:
one_count = data.pivot_table(index='street', values='name', aggfunc='count').sort_values(by='name')
one_count.columns = ['count']
one_count = one_count[one_count['count'] == 1]
one_count

На 458 улицах находится одно заведение

In [ ]:
one_count_st = one_count.index
one_count_data = data.query('street in @one_count_st')
one_count_data

In [ ]:
one_count_data = one_count_data.pivot_table(index='district', columns='category', values='rating', aggfunc='mean')
plt.figure(figsize=(14,6))
plt.title('Рейтинг по категориям')
sns.heatmap(one_count_data, annot=True)
plt.show()

Несмотря на то, что у людей нет альтернатив, рейтинги все равно очень разнообразны. Есть как хорошие, так и не очень заведения

In [ ]:
one_count_st = one_count.index
one_count_data = data.query('street in @one_count_st')
len(one_count_data.query('chain == 0'))

In [ ]:
len(one_count_data.query('chain == 1'))

В основном это не сетевые заведения

Найдем средний чек по району, так как в данных есть очень большие значения, будем использовать медиану

In [ ]:
middle_bill = data.groupby('district', as_index=False)['middle_avg_bill'].agg('median')
middle_bill

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['category']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=middle_bill,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlGnBu',
    fill_opacity=0.8,
    legend_name='Средний чек по районам',
).add_to(m)



# выводим карту
m

Центральный административный округ и Западный административный округ самые дорогие

## Открытие кофейни

In [ ]:
coffee_count = len(data[data['category'] == "кофейня"])
coffee_count

Всего в Москве 1413 кофейнь

In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data[data['category'] == "кофейня"].apply(create_clusters, axis=1)

# выводим карту
m

- Больше всего кофеен в центре, поэтому там открываться не выгодно
- Меньше всего кофеен в Южном,Юго-Восточном округе и Северо-Западных округах

In [ ]:
coffee = data[data['category'] == "кофейня"]

In [ ]:
rating_coffee = coffee.groupby('district', as_index=False)['rating'].agg('mean')
rating_coffee

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['category']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
coffee.apply(create_clusters, axis=1)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=rating_coffee,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGnBu',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг кофеен по районам',
).add_to(m)



# выводим карту
m

В Западном округе рейтинг ниже Москвы, но при этом достаточно много заведений.

In [ ]:
len(coffee[coffee['is_24/7']==True])

In [ ]:
is_24_coffee = coffee.groupby('is_24/7').agg({'name': 'count'})
is_24_coffee = is_24_coffee['name']
name= ['не 24/7','24/7']
fig = go.Figure(data=[go.Pie(labels=name, values=is_24_coffee,title='Распределение кофеен по графику работы')])
fig.show() 

59 кофеен работают 24/7, это всего 4% от общего числа

In [ ]:
middle_avg_bill_coffee = coffee.groupby('district', as_index=False)['middle_coffee_cup'].agg('mean')
middle_avg_bill_coffee

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['category']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
coffee.apply(create_clusters, axis=1)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=middle_avg_bill_coffee,
    columns=['district', 'middle_coffee_cup'],
    key_on='feature.name',
    fill_color='YlGnBu',
    fill_opacity=0.8,
    legend_name='Средняя стоимость чашки кофе по районам',
).add_to(m)



# выводим карту
m

- Самое дорогое кофе на Западе, Юго-Западе и в центре
- Самое дешевое на Востоке и Юго-Востоке

In [ ]:
coffee['middle_coffee_cup'].hist(bins=50)

Чашка кофе обойдется в сумму от 50 до 300 рублей

При открытии стоит ориентироваться на цену по району. Рассмотрим 3 района, где меньше всего кофеен, а следовательно и конкуренция ниже.

In [ ]:
south_district = coffee.query('district == "Южный административный округ"')
south_eastern_district = coffee.query('district == "Юго-Восточный административный округ"')
north_west_district = coffee.query('district == "Северо-Западный административный округ"')


In [ ]:
south_district['middle_coffee_cup'].hist(bins=15)

В южном округе чашечка кофе обойдется от 60 до 300 рублей

In [ ]:
south_eastern_district['middle_coffee_cup'].hist(bins=15)

В Юго-Восточном 50 - 225

In [ ]:
north_west_district['middle_avg_bill'].hist(bins=15)

В Северо-Западном от 210 до 390 рублей

На мой взгляд, лучше открывать кофейню в Северо-Западном округе, так как
- Небольшое количество конкурентов, так как в этом районе кофеен немного
- Высокая цена чашечки кофе у конкурентов, что позволяет не занижать цены
- Так как в этом районе цена чашечки кофе начинается от 210 рублей, можно сделать чуть ниже, чтобы цены казались демократичнее
- Круглосуточные заведения не пользуются популярностью

Презентация <https://disk.yandex.ru/i/N7JMEMI3Cz4OIQ>